In [1]:
from langchain.llms import GooglePalm
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

import os
import json
import pandas as pd
import traceback

from dotenv import load_dotenv
import PyPDF2

In [2]:
load_dotenv()

True

In [3]:
google_api_key=os.getenv("GOOGLE_API_KEY")

In [4]:
llm=ChatGooglePalm(google_api_key=google_api_key,temperature=0.3)

d:\MCQ Generator Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
llm

ChatGooglePalm(client=<module 'google.generativeai' from 'd:\\MCQ Generator Project\\venv\\lib\\site-packages\\google\\generativeai\\__init__.py'>, google_api_key=SecretStr('**********'), temperature=0.3)

In [6]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs

1. No repetition of questions.
2. All questions align with the provided text.
3. Your response follows the RESPONSE_JSON format below.


### RESPONSE_JSON
{response_json}

"""

In [7]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"

    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"

    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"

    }
}

In [8]:
response_json=json.dumps(RESPONSE_JSON)

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

d:\MCQ Generator Project\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
PATH="./../data.txt"

with open(PATH,'r') as f:
    TEXT=f.read()

In [12]:
NUMBER=5
SUBJECT='Biology'
TONE='simple'


In [13]:
quiz_result = quiz_chain.run({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    })

d:\MCQ Generator Project\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research question

In [14]:
quiz_result

'Sure, here is a quiz of 5 multiple choice questions for Biology students in simple tone:\n\n1. **What is biology?**\n    * A. The scientific study of life.\n    * B. The study of plants.\n    * C. The study of animals.\n    * D. The study of the human body.\n\n2. **What are some of the unifying themes in biology?**\n    * A. All organisms are made up of cells.\n    * B. All organisms process hereditary information encoded in genes.\n    * C. All organisms evolve.\n    * D. All of the above.\n\n3. **What is energy processing?**\n    * A. The process by which organisms obtain and use energy.\n    * B. The process by which organisms convert food into energy.\n    * C. The process by which organisms produce energy.\n    * D. The process by which organisms store energy.\n\n4. **What is homeostasis?**\n    * A. The ability of an organism to maintain a stable internal environment.\n    * B. The ability of an organism to regulate its own internal environment.\n    * C. The ability of an organ

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:

"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject','quiz'],template=TEMPLATE2)

In [17]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [18]:
review_result=review_chain.run({
    "subject":SUBJECT,
    "quiz":quiz_result
})



> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
Sure, here is a quiz of 5 multiple choice questions for Biology students in simple tone:

1. **What is biology?**
    * A. The scientific study of life.
    * B. The study of plants.
    * C. The study of animals.
    * D. The study of the human body.

2. **What are some of the unifying themes in biology?**
    * A. All organisms are made up of cells.
    * B. All organisms process hereditary information encoded in genes.
    * C. All organisms evolve.
    * D. All of the

In [19]:
review_result

"The quiz is well-written and easy to understand. The questions are clear and concise, and the answers are accurate and complete. The quiz is also appropriate for the level of students it is intended for. The only suggestion I would make is to add a few more questions to make it more challenging.\n\nHere are some additional questions that could be added:\n\n6. What are the levels of biological organization?\n7. What are the different types of cells?\n8. What are the different types of tissues?\n9. What are the different types of organs?\n10. What are the different types of systems?\n\nThese questions would help to assess students' understanding of the basic concepts of biology."

In [20]:
generate_evaluation_chain=SequentialChain(chains=[quiz_chain,review_chain],
                input_variables=["text","number","subject","tone","response_json"],
                output_variables=['quiz','review'],
                verbose=True)

In [21]:
PATH="./../data.txt"

In [22]:
with open(PATH,'r') as file :
    TEXT=file.read()

In [23]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [24]:
# TEXT
NUMBER=5
SUBJECT="Biology"
TONE='simple'
# RESPONSE_JOSN=RESPONSE_JOSN

In [25]:
# json.dumps(RESPONSE_JSON)

In [26]:

with get_openai_callback() as cb:
    response=generate_evaluation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )






> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

d:\MCQ Generator Project\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
Here is a quiz of 5 multiple choice questions for Biology students in simple tone, based on the provided text:

1. Biology is the scientific study of:
    * Life
    * Non-living things
    * The universe
    * The human body

2. All organisms are made up of:
    * Cells
    * Atoms
    * Molecules
    * Tissues

3. Energy processing is important to life because it allows organisms to:
    * Move
    * Grow
    * Reproduce
    * All of the above

4. Bio

In [27]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [28]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [29]:
quiz=response.get('quiz')

In [30]:
quiz

'Here is a quiz of 5 multiple choice questions for Biology students in simple tone, based on the provided text:\n\n1. Biology is the scientific study of:\n    * Life\n    * Non-living things\n    * The universe\n    * The human body\n\n2. All organisms are made up of:\n    * Cells\n    * Atoms\n    * Molecules\n    * Tissues\n\n3. Energy processing is important to life because it allows organisms to:\n    * Move\n    * Grow\n    * Reproduce\n    * All of the above\n\n4. Biologists study life at multiple levels of organization, including:\n    * The molecular level\n    * The cellular level\n    * The organismal level\n    * All of the above\n\n5. Life on Earth emerged more than:\n    * 3.7 billion years ago\n    * 4.5 billion years ago\n    * 5.4 billion years ago\n    * 6.3 billion years ago\n\nThe correct answers are:\n1. Life\n2. Cells\n3. All of the above\n4. All of the above\n5. 3.7 billion years ago\n\nI hope this helps!'

In [42]:
# Parse the questions, choices, and answers
questions = []
choices = []
answers = []




In [43]:
lines = quiz.split('\n')
i = 0
while i < len(lines):
    line = lines[i].strip()
    if line and line[0].isdigit():
        question = line.split(':')[0].strip()
        choice = ""
        i += 1
        while i < len(lines) and lines[i].strip().startswith("*"):
            choice += lines[i].strip() + " | "
            i += 1
        questions.append(question)
        choices.append(choice.strip(" | "))
    else:
        i += 1

In [44]:
questions=questions[:5]

In [45]:
choices=choices[:5]

In [46]:
answers_section = lines.index("The correct answers are:") + 1


In [47]:
for line in lines[answers_section:]:
    if line.strip():
        parts = line.strip().split(". ")
        if len(parts) == 2:  # Ensure the split was successful
            q_num, answer = parts
            answers.append(answer)


In [48]:
answers

['Life',
 'Cells',
 'All of the above',
 'All of the above',
 '3.7 billion years ago']

In [49]:
data={
    "MCQ":questions,
    "Choices":choices,
    "Answers":answers
}

df = pd.DataFrame(data)

In [50]:
df.head()

,MCQ,Choices,Answers
0,1. Biology is the scientific study of,* Life | * Non-living things | * The universe ...,Life
1,2. All organisms are made up of,* Cells | * Atoms | * Molecules | * Tissues,Cells
2,3. Energy processing is important to life beca...,* Move | * Grow | * Reproduce | * All of the a...,All of the above
3,4. Biologists study life at multiple levels of...,* The molecular level | * The cellular level |...,All of the above
4,5. Life on Earth emerged more than,* 3.7 billion years ago | * 4.5 billion years ...,3.7 billion years ago


In [87]:
df.to_csv('biology_question.csv',index=False)

In [88]:
from datetime import datetime

datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'06_26_2024_16_47_46'